<div style="text-align: center;">
<!-- <img src="jupiter.jpg" style="width:150px; border-radius:100px" /> <br /> -->
<h1 style="text-align: center; width:100%">Projet Analyse de Données</h1>
    Equipe : <a href="mailto:Othmene.BENAZIEB@ecam-strasbourg.eu">Othmène Benazieb</a>, <a href="mailto:Bounphathay.CHANTHASAY@ecam-strasbourg.eu">Bounphathay Chanthasay</a>, <a href="mailto:lefoulervincent@gmail.com">Vincent Le Fouler</a> <br />

    Formateur : <a href="mailto:manuel.simoes@cpc-analytics.fr">Manuel Simoes</a>

</div>






<style>
div.warn {    
    background-color: #fcf2f2;
    border-color: #dFb5b4;
    border-left: 5px solid #dfb5b4;
    padding: 0.5em;
    }
</style>

<div style="text-align: center;">

<h5 style="text-align: center; width:100%">Développement d'un modèle de machine learning pour détecter les tumeurs cancéreuses dans le cadre du cancer du sein.</h5>

</div>

### 1. Importation des librairies générales  

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### 2. Chargement des données

In [ ]:
raw_data = pd.read_csv('breast_cancer.csv')

### 3. Traitement des données
Analyse et exploration des données.
* Traitement des NaN
* Encodage de la target
* Sélection des features
#### 3.1 Affichage des informations

In [ ]:
raw_data.info()

D'après la méthode .info(), seule la colonne 32 compte des NaN. Les autres colonnes comportent chacune 569 valeurs non nulles. 

#### 3.2 Suppression de la colonne vide 32
La colonne 32 "Unname: 32" est une colonne totalement vide. Celle-ci est supprimée.

In [ ]:
raw_data.drop(columns = ['id', 'Unnamed: 32'], axis = 1, inplace = True)

#### 3.3 Encodage de la target
La target "diagnosis" est encodé de manière à avoir 0 pour la valeur M et 1 pour la valeur B.

In [ ]:
raw_data.replace({'B': 0, 'M': 1}, inplace = True)

#### 3.4 Matrice de corrélation
L'objectif est d'avoir une vue générale sur les données, de savoir qu'elles sont les features corrélées à la target et qu'elles sont les features corrélées entre elles. Et ensuite, de sélectionner les features les plus intéressantes en conséquence.

In [ ]:
# Fonction affichant une matrice de corrélation sur un set de données.
def corr_matrix(data, l = 25):
    corr = data.corr()
    plt.figure(figsize=(l,l))
    sns.heatmap(corr, cmap='coolwarm', linecolor='white', annot=True)
    plt.show()  
    
corr_matrix(raw_data)

In [ ]:
# Pour un jeu de données, une target et un seuil, on retourne un dataframe ne contenant que les features corrélées à plus de x% avec la target choisie.
def features_selection(raw_data, target, x):
    corr = raw_data.corr()
    names = corr[(corr[target] > x)].index
    filtered_data = pd.DataFrame()
    for i in names:
        filtered_data = pd.concat([filtered_data, raw_data[i]], axis = 1)
    return filtered_data

In [ ]:
# Application de notre fonction avec un seuil de 0.6 
filtered_data = features_selection(raw_data, 'diagnosis', 0.75)

In [ ]:
corr_matrix(filtered_data, l = 8) 

Après une application de la fonction features_selection avec un seuil de 0.6, on conserve 10 features.

#### 3.5 Features corrélées entre elles

Dans un second temps, on retire les features corrélées entre elles.

In [ ]:
# filtered_data.drop(columns = ['radius_mean', 'area_mean', 'radius_worst', 'area_worst'], inplace = True, axis = 1)
filtered_data.drop(columns = ['radius_worst', 'concave points_mean'], inplace = True, axis = 1)

In [ ]:
corr_matrix(filtered_data, l = 6) 

Après exploration des données, on conserve 6 features pour poursuivre l'étude.

In [ ]:
sns.countplot(filtered_data['diagnosis'], label = 'Count')
B, M = filtered_data['diagnosis'].value_counts()
print('Number of Benign: ',B)
print('Number of Malignant : ',M)

Il y a un nombre raisonnable de cas malins par rapport au nombre de cas bénins. Autrement, si par exemple le nombre de cas malins était très faible, notre futur algorithme aurait toujours prédit un résultat bénin, ayant considéré que la probabilité d'un cas malin était très faible.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
filtered_data.drop('diagnosis', axis = 1).hist(bins = 50 , figsize = (15,5))
plt.show()

In [ ]:
sns.pairplot(filtered_data, hue = 'diagnosis')

In [ ]:
# Séparation des sets, première opération à faire sur les données
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(filtered_data.drop('diagnosis', axis = 1), filtered_data['diagnosis'], test_size = 0.2, random_state = 10)

#### 3.6 Normalisation des données 
Plusieurs méthodes de normalisation vont être utilisées dans le but de les comparer et de garder la plus optimale. 

In [ ]:
# Importation des différentes méthodes de normalisation
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer

In [ ]:
# Fonction permettant de retourner dans un dictionnaire les données normalisées suivant différentes méthodes de normalisation.
# La clé représente le nom de la méthode de normalisation. Sa valeur les données normalisées par elles.
def scale_data(data):
    sc_data = {'Raw' : data}
    noms = ['StandardScaler', 'MinMaxScaler', 'RobustScaler', 'Normalizer']
    scalers = [StandardScaler(), MinMaxScaler(), RobustScaler(), Normalizer()]
    sc_data[noms[0]] = data
    for i in range(0, len(scalers)):
        sc = scalers[i]
        sc_data[noms[i]] = sc.fit_transform(data)
    return sc_data

In [ ]:
sc_data = scale_data(X_train)

#### 3.7 Comparaison de différents modèles

In [ ]:
# Cross-validation
from sklearn.model_selection import cross_val_predict

# Modèles de classification
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Métriques
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
def train_validation(X_train, y_train):
    result_table = pd.DataFrame(np.empty((0, 6)))
    result_table.columns = ['Normalization','Model', 'Precision', 'Recall', 'F1', 'ROC']
    for k, v in X_train.items():
        # Séparation des données en un jeu de train et de test
        # X_train, X_test, y_train, y_test = train_test_split(v, y, test_size = 0.2, random_state = 10)

        noms = ['SGDClassifier', 'LinearSVC', 'KNeighborsClassifier', 'RandomForestClassifier']
        classifier = [SGDClassifier(random_state = 10), LinearSVC(), KNeighborsClassifier(), RandomForestClassifier()]

        for i in range(0, len(classifier)):
            cl = classifier[i]
            # cl.fit(X_train, y_train)
            # On effectue une cross validation. 
            y_pred = cross_val_predict(cl, v, y_train, cv = 5)
            precision = round(precision_score(y_train, y_pred), 2)
            recall = round(recall_score(y_train, y_pred), 2)
            f = round(f1_score(y_train, y_pred), 2)
            roc = round(roc_auc_score(y_train, y_pred), 2)
            df_new_line = pd.DataFrame([[k, noms[i], precision, recall, f, roc]], columns=['Normalization','Model','Precision', 'Recall', 'F1', 'ROC'] )
            result_table = pd.concat([result_table, df_new_line], ignore_index=True)
    return result_table

In [ ]:
results = train_validation(sc_data, y_train)

In [ ]:
results.sort_values(by = ['Recall','Precision','F1'], ascending = False)

Dans la mesure où nous avons réduit le nombre de dimension à deux features, l'intérêt d'une PCA est inexistant. C'est pour cette raison qu'elle n'a pas été testé dans le cadre du projet. 

D'après l'ensemble de nos essais, nous pouvons conlure que la méthode de standardisation des données importe peu, sauf dans le cas du Normalizer() qui offre des résultats bien moindre que les autres méthodes. Pour ce qui est des modèles, le LinearSVC() et le KNClassifier() semblent équivalent. En revanche, le SGDClassifier offre de moins bons résultats.

Par conséquent, une étude approfondie sur le modèle LinearSVC() et KNeighborsClassifier() va être réalisées par la suite dans le but de sélectionner les meilleurs hyperparamètres du meilleur des deux modèle pour la résolution de notre problématique. 

### 4. Utilisation du GridSearchCV

In [ ]:
# sc_data_MinMax = sc_data['MinMaxScaler']
sc_data_MinMax = sc_data['StandardScaler']

In [ ]:
from sklearn.model_selection import GridSearchCV

#### 4.1 LinearSVC

In [ ]:
# Différents hyperparamètres de l'algorithme sont modulés, de façon à avoir in fine, les meilleurs paramètres.
param_grid = [
    {'loss' : ['hinge', 'squared_hinge'],
     'C' : range(1,100)
    }
    ]

cl = LinearSVC(max_iter = 1000000)

grid_search_linearSVC = GridSearchCV(cl, param_grid, cv = 5, scoring = 'recall', return_train_score = True)

grid_search_linearSVC.fit(sc_data_MinMax, y_train)
print(grid_search_linearSVC.best_estimator_)
print(grid_search_linearSVC.best_score_)

#### 4.2 KNeighborsClassifier

In [ ]:
# Différents hyperparamètres de l'algorithme sont modulés, de façon à avoir in fine, les meilleurs paramètres.
param_grid = [
    {'n_neighbors' : range(1,25),
     'algorithm' : ['ball_tree', 'kd_tree', 'brute'],
     'metric' : ['euclidean', 'manhattan']
    }
    ]

cl = KNeighborsClassifier()

grid_search_knn = GridSearchCV(cl, param_grid, cv = 5, scoring = 'recall', return_train_score = True)

grid_search_knn.fit(sc_data_MinMax, y_train)
print(grid_search_knn.best_estimator_)
print(grid_search_knn.best_score_)

Le modèle LinearSVC offre le meilleur recall. 
Pour cette raison, il sera sélectionné comme algorithme pour notre modèle final.

In [ ]:
final_model = grid_search_linearSVC.best_estimator_

### 5. Test du modèle final 

In [ ]:
sc = MinMaxScaler()
test_scale_data = sc.fit_transform(X_test)

In [ ]:
final_predictions = final_model.predict(test_scale_data)

#### 5.1 Résultats sur les métriques 

In [ ]:
precision = round(precision_score(y_test, final_predictions), 2)
recall = round(recall_score(y_test, final_predictions), 2)
f = round(f1_score(y_test, final_predictions), 2)
roc = round(roc_auc_score(y_test, final_predictions), 2)

print('-----------------------------------')
print('Final model results')
print('-----------------------------------')
print('Accuracy : ', precision * 100, ' %')
print('Recall : ', recall * 100, ' %')
print('F1 : ', f * 100, ' %')
print('Auc ROC : ', roc)
print('-----------------------------------')

#### 5.2 Résultats de la matrice de confusion

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, final_predictions)
sns.heatmap(cm, annot = True, cmap = 'Blues_r')

### 6. Sauvegarde du modèle et du scaler 

In [ ]:
import pickle

In [ ]:
# Sauvegarde du modèle
filename = 'final_model.sav'
pickle.dump(final_model, open(filename, 'wb'))

In [ ]:
# Sauvegarde du scaler
filename = 'sc_std.sav'
pickle.dump(sc, open(filename, 'wb'))